In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import load_model
import numpy as np

In [4]:
path_to_model = 'models/pnet.h5'
model = load_model(path_to_model)

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [9]:
tflitemodel = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/43/5blqqb3n63j88pcfn03dqgcm0000gn/T/tmptj5bc9m7/assets


\xbe\xef\x07\xc8\xbe\xc2\xbe\x9f\xbfH\xc8\x9b\xbe\xa4\xcc\xfc;\xe8\x15P\xbd,4?\xbd\xe4\x9b\xef<\xe2\x1b:>\x9e\x18.\xbd~o,\xbd\xf5g|=\xce\x996>pT\xd9:\x1e\xa0\x1f>\xfc\xb3\xa2\xbd/^\xae<\xce\xbd\xef<\x04\xdf\xc5\xbd\xc5*$>\x81wg=\x8d\xcaq=K\x1b.>\xe1R\xd9=\x1a\xe2Q\xbd*\x82\xc2\xbd\x00\xbeB\xbe\x1b\xe1(>kG\x94\xbc\x88a\x17\xbe\x19\x81M<\xee\xf2\x9b\xbc\xa51\xb2=C\xb5\x08>\xa0\x14)\xbe\xe3B=\xbe;\xd4\x17>\xa2\xe9\xc8>\xc1h\xef=\xad0V\xbc\x19\xfc\xc9\xbd^\xd7\xc0\xbc\xe0\xe4\x89>\xfb\xd9:>\xb7\x80\xe5=\x1a\xb7m\xbe\x04\xda\x03>VA\xaf>Y\xa8R>\xdb^@>\x93E4\xbd\x94\xd6\x9e>;\xb3\xab>-d\x9c\xbdp\xf3(=\xc5+\x92\xbe\xdb\x9e\x86<c\x02\xa3>\xe8\x10\x96>l?\x0c>YY\x89\xbe\x11\xa0\xbe=\x0e]\xd3<\xaf\xdd\xa4>T\xc7\xf4>\xb0\xf9\x05\xbe\xa3+\x87>4\xbb\n?\xb9U\xf4=a\xfb\xb0\xbd\x97\xca|\xbe<y\xf4\xbd\xd0\xf1M\xbe\rK&=\x8ad\xb8>\xedG\xb5=\xd7*\x8c\xbe/ta=3\x19]=\x93\xc3\xad>\xfb\xe1\xf8\xbe\n\x92b>\xd6\xbf\x8e\xbdR&\xce=\xcdQ\x01?\x89\x06\x80>7u\x06\xbc\xda\xd6\xda\xbe\xfd\x02\xa0\xbe3\xf4\xd8>X>\xc7\xbc